# 부동산 정보 크롤링 
## 데이터 출처 조인스

http://price.joinsland.joins.com/

http://price.joinsland.joins.com/area/index.asp?mcateGroup=A1&mcateCode=A1A3A4&areaCode=1168010600

<img width="80%" src="http://i.imgur.com/2i5uO2g.png" > 

화면 하단에 다음과 같은 데이터가 테이블(table) 표시된다. 이 데이터들이 크롤링하고자 하는 데이터

<img width="80%" src="http://i.imgur.com/ZZHv319.png" > 

여기에 표시되는 데이터가 어떻게 구성되어 있는지 개발자 도구로 살펴보자

개발자 도구에서 보면, id="sesae_info_gu" 인 div 태그 하위에 table이 있고, 여기에 데이터들이 구성되어 있음을 확인할 수 있다

<img width="80%" src="http://i.imgur.com/DWLXbjc.png" > 


하지만 pandas.read_html()로 읽었을 때 테이블이 읽히지 않는다.

```python
import pandas as pd

url = 'http://price.joinsland.joins.com/area/index.asp?mcateGroup=A1&mcateCode=A1A3A4&areaCode=1168010600'
dfs = pd.read_html(url, 'lxml')
dfs 
```
"ValueError: No tables found" 에러가 발생한다

# 페이지 소스 확인

메뉴에서 "보기 > 개발자 보기 > 소스보기" 선택하여 페이지 소스를 확인

크롬 개발자 도구로 table과 데이터가 보이지만, 소스 보기를 하면 소스에는 데이터가 보이지 않는다

id="sesae_info_gu" 인 div 태그에 table 도 없고 데이터도 없다

<img src="http://i.imgur.com/hzJFzQ1.png" >

이 처럼 크롬개발자 도구에서는 데이터가 있으나, 페이지 소스에는 데이터가 없는 경우는 다음 두 가지 중의 하나
* Ajax로 데이터를 요청하여 데이터를 표현 &rarr; 데이터 요청 URL을 따로 분석
* JavaScript가 실행되어 데이터를 표현 &rarr; Selenium 사용

Ajax (Asynchronous JavaScript and XML, 에이잭스)는 자바스크립트로 요청하는 비동기 데이터 요청.

# Ajax 데이터 요청 확인
* Network 탭을 열고
* 검색조건이 달라질 때('동'을 선택), 어떤 데이터가 오가는지 살펴본다
* 요청/응답 데이터들 중에 데이터기 포함되어 오가는지 살펴본다

<img src="http://i.imgur.com/5AG2YCg.png" >
    

# joins.com/ajax
여기서는 다음 URL로 데이터가 요청/응답 되었음을 확인할 수 있다

http://price.joinsland.joins.com/ajax/price.info.dongapt.asp?mcateGroup=A1&mcateCode=A1A3A4&areaCode=1168010600&danjiId=&ptype=&cpId=&aptamt=&aptarea=&aptcnt=&aptIpjudate=

<img width="70%" src="http://i.imgur.com/wAzM0TP.png">

    

다음 URL의 인자로 다음 값들이 전달.

http://price.joinsland.joins.com/ajax/price.info.dongapt.asp
* mcateGroup: A1=아파트, A6=오피스텔
* mcateCode: A1A3A4
* areaCode=1168010600

결과적으로 지역코드(areaCode)만 지정하면 가격  데이터를 가져올 수 있다.

# 지역이름과 코드
(참고) joinsland.joins.com 자체에도 법정동을 조회하는 내용이 있으나 데이터가 온전한 JSON 이 아니어서 가공하기 어렵다

In [1]:
# 구/시/군 (MCODE) 코드 가져오기

import requests

# 서울특별시 하위 '구/시/군' 읽기
area_code = '110000000' 
url = 'http://price.joinsland.joins.com/ajax/area_search.asp?div=MCODE&areaCode=' + area_code
r = requests.get(url)
print(r.text)

# JSON처럼 보이지만 JSON 규격을 따르지 않고 있다

[{seq:'1', idx:1, name:'강남구', label:'강남구', value:'1168000000', cnt:'0'},{seq:'2', idx:2, name:'강동구', label:'강동구', value:'1174000000', cnt:'0'},{seq:'3', idx:3, name:'강북구', label:'강북구', value:'1130500000', cnt:'0'},{seq:'4', idx:4, name:'강서구', label:'강서구', value:'1150000000', cnt:'0'},{seq:'5', idx:5, name:'관악구', label:'관악구', value:'1162000000', cnt:'0'},{seq:'6', idx:6, name:'광진구', label:'광진구', value:'1121500000', cnt:'0'},{seq:'7', idx:7, name:'구로구', label:'구로구', value:'1153000000', cnt:'0'},{seq:'8', idx:8, name:'금천구', label:'금천구', value:'1154500000', cnt:'0'},{seq:'9', idx:9, name:'노원구', label:'노원구', value:'1135000000', cnt:'0'},{seq:'10', idx:10, name:'도봉구', label:'도봉구', value:'1132000000', cnt:'0'},{seq:'11', idx:11, name:'동대문구', label:'동대문구', value:'1123000000', cnt:'0'},{seq:'12', idx:12, name:'동작구', label:'동작구', value:'1159000000', cnt:'0'},{seq:'13', idx:13, name:'마포구', label:'마포구', value:'1144000000', cnt:'0'},{seq:'14', idx:14, name:'서대문구', label:'서대문구', value:'1141000000', 

# 법정동 이름과 코드
부동산 정보등에는 법정동 코드가 사용된다. 
* 법정동 코드(8자리)
* 2(시/도) + 2자리(구/군/구) + 2자리(읍/면/동) + 2자리(리/단지)

법정동 전체 코드는 아래 링크

* https://goo.gl/tM6r3v

상세한 내용은 다음 링크에 설명을 참고

* http://nbviewer.jupyter.org/urls/financedata.github.io/posts/korea-area-code.ipynb

# 여기서부터 시작
- 위에서 본 것처럼 Joins 자체에 있는 법정동을 조회하지 않고 아래 사이트에 정리되어 있는 것을 이용해서 사용.

In [10]:
import pandas as pd
df_areacode = pd.read_csv('https://goo.gl/tM6r3v', sep='\t', dtype={'법정동코드':str, '법정동명':str})
df_areacode.head()

법정동코드           법정동명 폐지여부
0  1100000000          서울특별시   존재
1  1111000000      서울특별시 종로구   존재
2  1111010100  서울특별시 종로구 청운동   존재
3  1111010200  서울특별시 종로구 신교동   존재
4  1111010300  서울특별시 종로구 궁정동   존재

In [11]:
df_areacode.dtypes

법정동코드    object
법정동명     object
폐지여부     object
dtype: object

In [12]:
### 메인 코드 시작 ###

import pandas as pd

def get_areacode():
    df_areacode = pd.read_csv('https://goo.gl/tM6r3v', sep='\t', dtype={'법정동코드':str, '법정동명':str})
    df_areacode = df_areacode[df_areacode['폐지여부'] == '존재']
    df_areacode = df_areacode[['법정동코드', '법정동명']]
    return df_areacode

def get_province():
    df_areacode = get_areacode()
    df_province = df_areacode[ df_areacode['법정동코드'].str.contains('\d{2}0{8}|36110{6}')]   # \d: digit(숫자)
                                                                                               # 0{8}: 0 이 8개
    return df_province

In [14]:
df_areacode = get_areacode(); df_areacode.head()

법정동코드           법정동명
0  1100000000          서울특별시
1  1111000000      서울특별시 종로구
2  1111010100  서울특별시 종로구 청운동
3  1111010200  서울특별시 종로구 신교동
4  1111010300  서울특별시 종로구 궁정동

In [16]:
df_province = get_province()
df_province

법정동코드     법정동명
0      1100000000    서울특별시
2615   2600000000    부산광역시
2912   2700000000    대구광역시
3244   2800000000    인천광역시
3579   2900000000    광주광역시
3825   3000000000    대전광역시
4009   3100000000    울산광역시
4241   3611000000  세종특별자치시
4391   4100000000      경기도
10867  4200000000      강원도
13666  4300000000     충청북도
16797  4400000000     충청남도
22050  4500000000     전라북도
25397  4600000000     전라남도
30169  4700000000     경상북도
39299  4800000000     경상남도
45741  5000000000  제주특별자치도

In [17]:
df_areacode.head(10)

법정동코드           법정동명
0  1100000000          서울특별시
1  1111000000      서울특별시 종로구
2  1111010100  서울특별시 종로구 청운동
3  1111010200  서울특별시 종로구 신교동
4  1111010300  서울특별시 종로구 궁정동
5  1111010400  서울특별시 종로구 효자동
6  1111010500  서울특별시 종로구 창성동
7  1111010600  서울특별시 종로구 통의동
8  1111010700  서울특별시 종로구 적선동
9  1111010800  서울특별시 종로구 통인동

In [18]:
df_areacode[ df_areacode['법정동명'].str.contains('서울특별시 강남구') ]

법정동코드            법정동명
1003  1168000000       서울특별시 강남구
1037  1168010100   서울특별시 강남구 역삼동
1039  1168010300   서울특별시 강남구 개포동
1040  1168010400   서울특별시 강남구 청담동
1041  1168010500   서울특별시 강남구 삼성동
1042  1168010600   서울특별시 강남구 대치동
1043  1168010700   서울특별시 강남구 신사동
1044  1168010800   서울특별시 강남구 논현동
1045  1168011000  서울특별시 강남구 압구정동
1046  1168011100   서울특별시 강남구 세곡동
1047  1168011200   서울특별시 강남구 자곡동
1048  1168011300   서울특별시 강남구 율현동
1049  1168011400   서울특별시 강남구 일원동
1050  1168011500   서울특별시 강남구 수서동
1053  1168011800   서울특별시 강남구 도곡동

# 데이터 전처리

http://price.joinsland.joins.com/ajax/price.info.dongapt.asp
* mcateGroup A1=아파트, A6=오피스텔
* mcateCode: A1A3A4
* areaCode=1168010600 # 강남구 대치동

In [19]:
import pandas as pd
url = 'http://price.joinsland.joins.com/ajax/price.info.dongapt.asp?mcateGroup=A1&mcateCode=A1A3A4&areaCode=' + '1168010600'
    
print(url)

http://price.joinsland.joins.com/ajax/price.info.dongapt.asp?mcateGroup=A1&mcateCode=A1A3A4&areaCode=1168010600


In [20]:
dfs = pd.read_html(url,encoding='utf-8')      # list

In [27]:
type(dfs), len(dfs)

(list, 1)

In [28]:
type(dfs[0])

pandas.core.frame.DataFrame

In [29]:
df = dfs[0]         # dataframe
df.head(10)

단지명 면적 (㎡)           매매가 (만원)  매물           전세가 (만원)  매물.1  \
0         개포우성1차(아파트)    102  245,000 ~ 265,000   0   95,000 ~ 105,000     0   
1         개포우성1차(아파트)    148  315,000 ~ 340,000   2  120,000 ~ 140,000     0   
2         개포우성1차(아파트)    181  362,500 ~ 400,000   0  135,000 ~ 155,000     0   
3         개포우성1차(아파트)    214  420,000 ~ 450,000   0  155,000 ~ 175,000     0   
4         개포우성2차(아파트)    101  245,000 ~ 265,000   0   95,000 ~ 105,000     1   
5         개포우성2차(아파트)    146  320,000 ~ 350,000   0  120,000 ~ 140,000     1   
6         개포우성2차(아파트)    180  362,500 ~ 400,000   0  135,000 ~ 155,000     0   
7  대치대우아이빌멤버스4차(주상복합)     31    25,000 ~ 27,000   0    20,000 ~ 23,000     4   
8  대치대우아이빌멤버스4차(주상복합)     39    28,500 ~ 30,750   1    22,250 ~ 25,000     0   
9  대치대우아이빌멤버스4차(주상복합)     44    33,750 ~ 36,500   2    26,500 ~ 29,000     1   

   문의처  
0  NaN  
1  NaN  
2  NaN  
3  NaN  
4  NaN  
5  NaN  
6  NaN  
7  NaN  
8  NaN  
9  NaN

<img width="80%" src="http://i.imgur.com/mEEA1PN.png" >

- 여기서는 html 소스에서 바로 데이터를 추출해본다.

In [33]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'http://price.joinsland.joins.com/ajax/price.info.dongapt.asp?mcateGroup=A1&mcateCode=A1A3A4&areaCode=' + '1168010600'

r = requests.get(url)

soup = BeautifulSoup(r.text, 'lxml')        # lxml is a xml parser
# soup = BeautifulSoup(r.text, 'html.parser') 

In [34]:
table = soup.find('table')
trs = table.tbody.find_all('tr')

In [37]:
len(trs), type(trs), trs[0]

(158, bs4.element.ResultSet, <tr>
 <td class="txt_danzi" rowspan="4" valign="top"><a href="/area/?mcateGroup=A1&amp;areaCode=1168010600&amp;danjiId=244">개포우성1차</a><br/>(아파트)</td>
 <td class="num_area"><a href="/area/?mcateGroup=A1&amp;areaCode=1168010600&amp;danjiId=244&amp;ptype=1" style="color:#6284d4">102</a></td>
 <td class="num_price">245,000 ~ 265,000 </td>
 <td class="num_maemul"><a href="javascript:goMaemul('1','244')">0</a></td>
 <td class="num_price">95,000 ~ 105,000</td>
 <td class="num_maemul"><a href="javascript:goMaemul('2','244')">0</a></td>
 <!--td class="check"><input name="" type="checkbox" value="" /></td-->
 <td class="txt_inquiry" rowspan="4" valign="top">
 </td>
 </tr>)

In [39]:
""" The <table> tag defines an HTML table.
- Each table row is defined with a <tr> tag. 
  Each table header is defined with a <th> tag. 
  Each table data/cell is defined with a <td> tag.
- By default, the text in <th> elements are bold and centered.
- By default, the text in <td> elements are regular and left-aligned.
"""

' The <table> tag defines an HTML table.\n- Each table row is defined with a <tr> tag. \n  Each table header is defined with a <th> tag. \n  Each table data/cell is defined with a <td> tag.\n- By default, the text in <th> elements are bold and centered.\n- By default, the text in <td> elements are regular and left-aligned.\n'

In [40]:
rowspan_val = ''

for tr in trs[:20]: # 처음 20개 확인
    tds = tr.find_all('td')
    if tds[0].has_attr('rowspan'):    # rowspan -> row를 합칠때 사용
        rowspan_val = tds[0].text
        단지명, 면적, 매매가, 매물, 전세가, 전세_매물 = tds[0].text, tds[1].text, tds[2].text, tds[3].text, tds[4].text, tds[5].text
    else:
        단지명, 면적, 매매가, 매물, 전세가, 전세_매물 = rowspan_val, tds[0].text, tds[1].text, tds[2].text, tds[3].text, tds[4].text
    매매가_저 = int(매매가.split(' ~ ')[0].replace(',', ''))
    매매가_고 = int(매매가.split(' ~ ')[1].replace(',', ''))
    전세가_저 = int(전세가.split(' ~ ')[0].replace(',', ''))
    전세가_고 = int(전세가.split(' ~ ')[1].replace(',', ''))
    print(단지명, 면적, 매매가_저, 매매가_고, 매물, 전세가_저, 전세가_고, 전세_매물)

개포우성1차(아파트) 102 245000 265000 0 95000 105000 0
개포우성1차(아파트) 148 315000 340000 2 120000 140000 0
개포우성1차(아파트) 181 362500 400000 0 135000 155000 0
개포우성1차(아파트) 214 420000 450000 0 155000 175000 0
개포우성2차(아파트) 101 245000 265000 0 95000 105000 1
개포우성2차(아파트) 146 320000 350000 0 120000 140000 1
개포우성2차(아파트) 180 362500 400000 0 135000 155000 0
대치대우아이빌멤버스4차(주상복합) 31 25000 27000 0 20000 23000 4
대치대우아이빌멤버스4차(주상복합) 39 28500 30750 1 22250 25000 0
대치대우아이빌멤버스4차(주상복합) 44 33750 36500 2 26500 29000 1
대치대우아이빌멤버스4차(주상복합) 44A 33750 36500 2 26500 29000 0
대치대우아이빌멤버스4차(주상복합) 45B 33750 36500 0 26500 29000 1
대치대우아이빌멤버스4차(주상복합) 51A 34500 37500 0 27000 29000 0
대치대우아이빌멤버스4차(주상복합) 51B 34500 37500 0 27000 29000 0
대치대우아이빌멤버스4차(주상복합) 53 37000 40000 1 29500 32500 0
대치대우아이빌멤버스4차(주상복합) 58 39500 42500 0 32500 34500 0
대치대우아이빌멤버스4차(주상복합) 63 48500 51500 1 34500 36500 0
대치대우아이빌멤버스4차(주상복합) 68 48500 51500 0 34500 36500 0
대치대우아이빌명문가(주상복합) 41A 30500 32500 1 24500 26500 2
대치대우아이빌명문가(주상복합) 43B 30500 32500 2 24500 26500 1


In [42]:
# 참고 (한글로 변수 OK)
매매 = 500; 전세 = 90
print(매매+전세)

590


#  Dataframe 으로 만들기

In [44]:
# DataFrame으로 만들기
values_list = []

table = soup.find('table', attrs={'class':'tbl_compare'})
trs = table.tbody.find_all('tr')

rowspan_val = ''

for tr in trs:
    tds = tr.find_all('td')
    if tds[0].has_attr('rowspan'):
        rowspan_val = tds[0].text
        단지명, 면적, 매매가, 매물, 전세가, 전세_매물 = tds[0].text, tds[1].text, tds[2].text, tds[3].text, tds[4].text, tds[5].text
    else:
        단지명, 면적, 매매가, 매물, 전세가, 전세_매물 = rowspan_val, tds[0].text, tds[1].text, tds[2].text, tds[3].text, tds[4].text
    매매가_저 = int(매매가.split(' ~ ')[0].replace(',', ''))
    매매가_고 = int(매매가.split(' ~ ')[1].replace(',', ''))
    전세가_저 = int(전세가.split(' ~ ')[0].replace(',', ''))
    전세가_고 = int(전세가.split(' ~ ')[1].replace(',', ''))
    values_list.append([단지명, 면적, 매매가_저, 매매가_고, 매물, 전세가_저, 전세가_고, 전세_매물])
    
cols = ['단지명', '면적', '매매가_저', '매매가_고', '매물', '전세가_저', '전세가_고', '전세가_매물']   
df = pd.DataFrame(values_list, columns=cols)

print ("건수:", len(df))
df.head(20)    # 앞의 pd.read_html() 로 읽은 것과 포맷이 약간 다름.

건수: 158


단지명   면적   매매가_저   매매가_고 매물   전세가_저   전세가_고 전세가_매물
0          개포우성1차(아파트)  102  245000  265000  0   95000  105000      0
1          개포우성1차(아파트)  148  315000  340000  2  120000  140000      0
2          개포우성1차(아파트)  181  362500  400000  0  135000  155000      0
3          개포우성1차(아파트)  214  420000  450000  0  155000  175000      0
4          개포우성2차(아파트)  101  245000  265000  0   95000  105000      1
5          개포우성2차(아파트)  146  320000  350000  0  120000  140000      1
6          개포우성2차(아파트)  180  362500  400000  0  135000  155000      0
7   대치대우아이빌멤버스4차(주상복합)   31   25000   27000  0   20000   23000      4
8   대치대우아이빌멤버스4차(주상복합)   39   28500   30750  1   22250   25000      0
9   대치대우아이빌멤버스4차(주상복합)   44   33750   36500  2   26500   29000      1
10  대치대우아이빌멤버스4차(주상복합)  44A   33750   36500  2   26500   29000      0
11  대치대우아이빌멤버스4차(주상복합)  45B   33750   36500  0   26500   29000      1
12  대치대우아이빌멤버스4차(주상복합)  51A   34500   37500  0   27000   29000      0
13  대치대우아이빌멤버스4차(주상복합)  51B   34500   37500  0   27000   29000      0
14  대치대우아이빌멤버스4차(주상복합)   53   37000   40000  1   29500   32500      0
15  대치대우아이빌멤버스4차(주상복합)   58   39500   42500  0   32500   34500      0
16  대치대우아이빌멤버스4차(주상복합)   63   48500   51500  1   34500   36500      0
17  대치대우아이빌멤버스4차(주상복합)   68   48500   51500  0   34500   36500      0
18    대치대우아이빌명문가(주상복합)  41A   30500   32500  1   24500   26500      2
19    대치대우아이빌명문가(주상복합)  43B   30500   32500  2   24500   26500      1

# 함수로 만들기

In [57]:
# 조인스부동산 아파트 조회

import pandas as pd
import requests
from bs4 import BeautifulSoup

def get_areacode():
    df_areacode = pd.read_csv('https://goo.gl/tM6r3v', sep='\t', dtype={'법정동코드':str, '법정동명':str})
    df_areacode = df_areacode[df_areacode['폐지여부'] == '존재']
    df_areacode = df_areacode[['법정동코드', '법정동명']]
    return df_areacode

def get_province():
    df_areacode = get_areacode()
    df_province = df_areacode[ df_areacode['법정동코드'].str.contains('\d{2}0{8}|36110{6}')]
    return df_province


def joins_realasset(areacode=''):
    url = 'http://price.joinsland.joins.com/ajax/price.info.dongapt.asp?mcateGroup=A1&mcateCode=A1A3A4&areaCode=' + areacode

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    values_list = []

    table = soup.find('table', attrs={'class':'tbl_compare'})
    trs = table.tbody.find_all('tr')
    rowspan_val = ''

    for tr in trs:
        tds = tr.find_all('td')
        if tds[0].has_attr('rowspan'):
            rowspan_val = tds[0].text
            단지명, 면적, 매매가, 매물, 전세가, 전세_매물 = tds[0].text, tds[1].text, tds[2].text, tds[3].text, tds[4].text, tds[5].text
        else:
            단지명, 면적, 매매가, 매물, 전세가, 전세_매물 = rowspan_val, tds[0].text, tds[1].text, tds[2].text, tds[3].text, tds[4].text
        매매가_저 = int(매매가.split(' ~ ')[0].replace(',', ''))
        매매가_고 = int(매매가.split(' ~ ')[1].replace(',', ''))
        전세가_저 = int(전세가.split(' ~ ')[0].replace(',', ''))
        전세가_고 = int(전세가.split(' ~ ')[1].replace(',', ''))
        values_list.append([단지명, 면적, 매매가_저, 매매가_고, 매물, 전세가_저, 전세가_고, 전세_매물])

    cols = ['단지명', '면적', '매매가_저', '매매가_고', '매물', '전세가_저', '전세가_고', '전세가_매물']   
    df = pd.DataFrame(values_list, columns=cols)
    return df

# 검색과 활용

In [58]:
df_areacode = get_areacode()

In [59]:
area = '신사동'
area_code = df_areacode[ df_areacode['법정동명'].str.contains(area) ]
area_code

법정동코드           법정동명
582   1138010900  서울특별시 은평구 신사동
1043  1168010700  서울특별시 강남구 신사동

In [60]:
area = '강남구 신사동'
area_code = df_areacode[ df_areacode['법정동명'].str.contains(area) ]['법정동코드'].values[0]
area_code

'1168010700'

In [56]:
joins_realasset(area_code)

단지명    면적   매매가_저   매매가_고 매물   전세가_저   전세가_고 전세가_매물
0      대원칸타빌(아파트)    85   82500   92500  0   45000   49000      0
1      대원칸타빌(아파트)  107B  122500  135000  0   58500   62500      0
2      대원칸타빌(아파트)   123  127500  145000  0   62500   68000      0
3      대원칸타빌(아파트)   144  150000  167500  0   87500   97500      0
4      로데오현대(아파트)    73   75000   83000  0   39500   43500      0
5      로데오현대(아파트)   114  110000  125000  0   53500   57000      0
6      로데오현대(아파트)  122A  117500  130000  0   56500   61500      0
7         신성(아파트)    72   97000  103000  0   52500   57500      0
8         신성(아파트)    98  122500  140000  0   59000   68000      0
9   압구정하이츠파크(아파트)   234  321000  350000  0  210000  222500      0
10  압구정하이츠파크(아파트)   270  337500  366000  0  222500  242500      0
11      현대맨션(아파트)   147  201000  211000  0   88000  101000      0
12      현대맨션(아파트)   147  201000  211000  0   88000  101000      0
13      현대맨션(아파트)   159  226000  236000  0   96500  110000      0
14      현대맨션(아파트)   162  226000  236000  0   96500  110000      0
15      현대맨션(아파트)   191  223500  233500  0   99000  110000      0
16      현대맨션(아파트)   194  223500  233500  0   99000  110000      0
17      현대맨션(아파트)  195B  223500  233500  0   99000  110000      0
18      현대맨션(아파트)  213A  288500  299000  0  110000  125000      0
19      현대맨션(아파트)  212B  288500  299000  0  110000  125000      0
20      현대맨션(아파트)  213C  288500  299000  0  110000  125000      0
21      현대맨션(아파트)   218  288500  299000  0  110000  125000      0